# Sum_Links files exploration and processing
La idea es levantar atributos para el dataset estandar

* observar estructura de la tabla 
* levantar solo los pares (i,j) donde i & j son users de la telco. Para saber si son users de la teclo es necesario levantar el file del hash_map (user_hash -- > int). Descartando asi el resto de los users.
* extraer features de uso "endemico" para nuestros users de la TelCo

## definiciones utilizadas
* timestamp arranca en 0 segundos para 01/01/2012 00:00am
* La Direction, viene dada relativa al user de la primer columna (incoming si entra y viceversa si es outgoing).

### el dataset vendria con este header
 {'LineKeyOrigin':hash_object \\\  'LineKeyTarget':np.uint32 \\\  'CallsWeekDayLight':np.uint8 \\\   'CallsWeekNight':np.uint8 \\\  'CallsWeekend':np.uint8 \\\  'TimeWeekDaylight':np.uint8 \\\  'TimeWeekNight':np.uint8 \\\  'TimeWeekend':np.uint8 \\\  'SmsWeekDaylight':np.uint8 \\\  'SmsWeekNight ':np.uint8 \\\  'SmsWeekend':np.uint8} 

In [1]:
import pandas as pd; import numpy as np; import os;import random;
pd.set_option('display.max_rows', 300)
from matplotlib import pyplot as plt
%matplotlib inline
import unicodedata
import gzip
import os

In [2]:
import shutil

In [4]:
np.random.seed(2015)

In [5]:
antennas= pd.read_csv('antennas_mexico.csv',
                      sep = "|",
                      header=0,
                      index_col=0)
antennas.index.rename("AntennaID",inplace=True)

#en user_dict va a quedar el mapping de hashes por usuario a su int correspondiente.

users_dict= pd.read_csv('hash_user_dict_mexico.csv.gz',
                       sep = "|",
                       header=0,
                       index_col=0).to_dict()

In [28]:
test_input  = "/grandata/voice/201401_geo/GEO\ Voz\ Pospago\ 01-02012014.txt.gz"
!zcat $test_input

In [16]:
#seteamos el lugar de trabajo
rootdir="/grandata/voice/sum_links"
os.chdir(rootdir)
year = "2015"; 
month_start= "04";
#todos estos sum_links terminan 2 meses mas tardes
month_end= "0"+str(int(month_start)+2)
input_file= rootdir +"/sum_links_{y}{ms}_{y}{me}.txt.gz".format(y=year,ms=month_start,me=month_end)

In [19]:
input_file

'/grandata/voice/sum_links/sum_links_201504_201506.txt.gz'

In [20]:
!zcat $input_file | head -n10

LineKeyOrigin|LineKeyTarget|CallsWeekDaylight|CallsWeekNight|CallsWeekend|TimeWeekDaylight|TimeWeekNight|TimeWeekend|SmsWeekDaylight|SmsWeekNight|SmsWeekend
CD5844F7EB2C00C63465E43B44EB9025|3F6FEB3BA73D7E51D906B19F1E0A9099|0|0|0|0|0|0|1|0|0
6AEE0F059A39A7E1CF90B93B5BE8E5AD|3F6FEB3BA73D7E51D906B19F1E0A9099|0|0|0|0|0|0|0|0|3
CD5844F7EB2C00C63465E43B44EB9025|21F587020D72C34131DDD04C0FCD6CFF|0|0|0|0|0|0|0|0|1
CD5844F7EB2C00C63465E43B44EB9025|796D132ECDF757CBCE8FB01DDA3891D6|0|0|0|0|0|0|1|0|0
5E8BADC0F256DEC1ACBB1AB2B77527E9|796D132ECDF757CBCE8FB01DDA3891D6|0|0|3|0|0|203|0|0|0
CD5844F7EB2C00C63465E43B44EB9025|5FA3B22C222AFFF5DFAAE7CFCAEBCA30|0|0|0|0|0|0|0|0|1
6B56A956E1A9E56BE4D90543C8706E76|5FA3B22C222AFFF5DFAAE7CFCAEBCA30|0|0|0|0|0|0|1|0|0
2752F0E0F8226FFF2A3189EB131330AC|46B254E4C1A98CECB4F3D52E66BB836D|1|0|1|196|0|36|0|0|0
2752F0E0F8226FFF2A3189EB131330AC|A954CAD047A827B6DEBA10BD797E7B16|10|4|9|1403|208|1416|0|0|0

gzip: stdout: Broken pipe


## Asumimos...
* los calls son un conteo de quantity.
* el time se refiere a cantidad de segundos

In [ ]:
##leemos alguna tabla ejemplo
tabla2 = pd.read_csv(input_file,
                     sep = "|", 
              names = ['LineKeyOrigin', 'LineKeyTarget', 'CallsWeekDaylight', 'CallsWeekNight',
                       'CallsWeekend','TimeWeekDaylight', 'TimeWeekNight', 'TimeWeekend',
                       'SmsWeekDaylight', 'SmsWeekNight', 'SmsWeekend'],
                     usecols=
                             engine = 'c',index_col=None, lineterminator='\n', skipinitialspace=True, 
                    header=None,dtype = {'Target':np.int32,'Destination':np.uint32,'TimeStamp':np.uint32,
                                      'Duration':np.uint16,'AntennaID':np.uint16,'Direction':np.object_})

In [ ]:
tabla.dtypes, tabla2.dtypes
#tabla['Destination'] = tabla['Destination'].astype(np.uint32,copy=False);
#tabla['Target'] = tabla['Target'].astype(np.uint32,copy=False);
#tabla['TimeStamp'] = tabla['TimeStamp'].astype(np.uint32,copy=False);
#tabla['Duration'] = tabla['Duration'].astype(np.uint16,copy=False);
#tabla['AntennaID'] = tabla['AntennaID'].astype(np.uint16,copy=False);
#tabla['Direction'] = tabla['Direction'].astype(np.object_,copy=False);

In [ ]:
tabla.sort_values(by='TimeStamp',ascending=True,inplace=True)
tabla2.sort_values(by='TimeStamp',ascending=True,inplace=True)

In [ ]:
tabla2.sort_values(by='TimeStamp',ascending=True,inplace=True)

In [ ]:
#agarro una muestra de 300k para hacer pruebas
sample=tabla.ix[random.sample(tabla.index, 3*(10**5))]
sample2 = tabla.ix[random.sample(tabla.index, 3*(10**5))]

In [ ]:
#enriquezco el dataset con la info de antennas_mexico.csv
enriched_sample = pd.merge(sample,antennas,left_on="AntennaID",right_index=True)
enriched_sample2 = pd.merge(sample2,antennas,left_on="AntennaID",right_index=True)

In [ ]:
enriched_sample.sort_values(by='TimeStamp',ascending=True,inplace=True)
enriched_sample2.sort_values(by='TimeStamp',ascending=True,inplace=True)

In [ ]:
enriched_sample.head()

## Mobility
La idea es ahora mirar dentro de la base (prototipamos con sample) la lista de antenas utilizadas por cada usuario para obtener la variable dependiente $Y$ que vendria a ser si el tipo viajo o no por la zona endemica (visto como un llamado basicamente desde una antena en la epidemic zone). Despues buscaremos atributos, o variables regresoras que logren predecir ese comportamiento en el pasado pero usando solo los atributos en un timeframe "del futuro". Busco atributos que correlacionen con 'haber viajado por la zona endemica en el pasado'
#### pseudocode:
* tomamos el set de origins/targets de la tabla.
* p/c/user lo mapeamos a un conjunto de antenas que sabemos que el tipo uso en algun CDR. Aca usamos alguna operacion split, apply, combine dondeset donde el key va a ser la columna origin y la data es AntennaID. Ahi extraemos la lista de antenas que el user uso. 
* descartamos del dataset los users "vacios". i.e. aquellos que su conjunto de antenas da vacio. Para esto descartamos todas las filas que no tienen ningun user (origin o target) en la tabla split/apply/combine. Asi obtendremos una lista de usuarios de la telco que viajaron por mexico a partir de sus llamados en ese mes. Despues solo hacer un outer join de todas estas tablas resultantes para cada mes. Asi como tambien un join de estos "sets" de antennas por user. 


In [ ]:
grouped = tabla['AntennaID'].groupby(tabla['Target'])

In [ ]:
#con esto extraigo el mapping user --> antennas used that month
user_antenna_map = sample.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();
user_antenna_map2 = sample2.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();

#esto es 'tipo' un head para dicts
dict(user_antenna_map.items()[0:10]), dict(user_antenna_map2.items()[0:10])

from collections import defaultdict
user_antenna_map_concat = defaultdict(list)

#Veo como concatenar dos "maps" distintos y dejando solo las antenas unicas..
for d in (user_antenna_map, user_antenna_map2): # podemos meter cuantos dicts queramos aca..
    for key, value in d.iteritems():
        #list(set()) elimina repetidos
        user_antenna_map_concat[key] = list(set(user_antenna_map_concat[key] + value))


In [ ]:
## scripot para convertir las columnas de otros types --> ints

passes=5

#ver el tiempo que tarda
now = time.time()

#el chunk basicamente va leyendo el file de a 'chunksize' cantidad de filas
#subgroup = pd.DataFrame()

for group in range(passes):
    subgroup = pd.DataFrame()
    table = pd.read_csv(
            input_file,
            engine = 'c',
            chunksize = 5*10**7,
#            iterator =True,
            sep = ' ',
            header = None,
            index_col=None,
            names = ['Target','Destination','Direction','TimeStamp','Duration','AntennaID'],
            usecols = ['Target', 'AntennaID','TimeStamp'],
            dtype = {'Target':np.uint32,'Destination':np.uint32,'TimeStamp':np.uint32,
                                      'Duration':np.uint16,'AntennaID':np.uint16,'Direction':np.object_}
            )
    
    #cuando entramos a este loop, table tiene tantos 'chunks' como el valor entero de la cantidad de lineas en el file
    #dividido el tamanyo del chunksize
    for chunk in table:
            #a cada chunk filtro por todos los Targets que tengan resto group modulo passes. y trabajo sobre la tabla
            #subgroup nada mas que ahora tiene pocos usuarios
            chunk['Target_int'] = chunk[].map(users_dict)
            subgroup = subgroup.append(chunk[chunk['Target'] % passes == group])
    #
    #entonces la idea es que yo ahora solo voy a trabajar, dentro de la tabla
    
    #paso de segundos a horas
    #notar que timestamp arranca en 0 segundos para domingo 01/01/2012 00:00am 
    #con lo cual domingo es el dia 0, lunes el 1, asi..
    
    subgroup['Hour'] =  (subgroup['TimeStamp'].values*1.0/3600)%24
    subgroup['Day']  =  (subgroup['TimeStamp'].values*1.0/(3600*24))%7
    
    #filtro usuarios con pocos o demasiados llamados en general menos de 5 mensuales y mas de 400  
    insignificant_users = subgroup['Target'].value_counts()[(subgroup['Target'].value_counts() < 5) \
                        | (subgroup['Target'].value_counts() > 400)].index.values.tolist()
    subgroup= subgroup.loc[~subgroup['Target'].isin(insignificant_users)]
    
    
    #filtro segun nightfilter y week_end
    if night_filter == True:
        subgroup = subgroup.loc[(subroup['Hour']<8) | (subgroup['Hour']>18)]
    if week_end == True:
        subgroup = subgroup.loc[(subgroup['Day']==0) | (subgroup['Day']==6)]

    
    #   
        
    grouped = subgroup.groupby(['Target', 'AntennaID'])['AntennaID'].agg({'count': np.size})
    grouped.reset_index(inplace=True,drop=False)
    
    del subgroup
    #reordeno dentro de c/ target por el count del antenna, esto me sirve para despues ordenar las antenas por uso
    grouped.sort_values(by=['Target','count'],ascending=False,inplace=True)
    
    ##enriquezco la muestra con datos epidemicos
    #primero agrego a cada antenna del df el dato de si es epidemica
    #despues agrupo por el target y me fijo solo la columna epidemica en c/grupo
    #finalmente sumo en c/ grupo y tomo la parte superior 
#entera de esa division con el largo del grupo. Si uso al menos una antena epidemica entonces esta expuesto(==1) Si no,
# da 0 pues no estuvo expuesto.        
    ##enriquezco la muestra con datos epidemicos

    exposed_info =grouped.join(antennas['EPIDEMIC'], on='AntennaID').\
    groupby('Target')['EPIDEMIC'].\
        agg({'EXPOSED' : lambda x: int( np.ceil(np.sum(x)*1.0/np.size(x)) )}) 
    
    #actualizo la tabla
    grouped = grouped.join(exposed_info['EXPOSED'],on="Target")
    del exposed_info
    
    
    #creo la tabla filtrada solo por users, que es la que voy a terminar guardando (hay tantos rows como users)
    output_table = grouped.drop_duplicates(subset = 'Target', keep='first')
    #re indexo
    output_table.index = output_table['Target'].values
    
    #agrupo ahora la tabla por target para hacer todos los calculos en los grupos
    
    grouped = grouped.groupby('Target')  
    
    #aca voy a ir agregando las top 10 antennas utilizadas por el user, Si no llego a 10 antennas, relleno con NaNs
    for i in range(0,10):
        #me quedo con la i-esima fila de c/grupo (si no hay fila, no toma en cuenta ese Target)
        
        buffer_table = grouped.nth(i)[['AntennaID','count']]
        #renombre a iesima AntennaId e iesimo count
        buffer_table.columns=['AntennaID_%i'%i,'count_%i'%i]
        #agrego esta info como nuevas columnas, dejando lo demas como NaNs
        output_table = pd.concat([output_table, buffer_table], axis=1, join_axes=[output_table.index])
    
    del grouped
    #los primeros AntennaIDs ya no me sirven, idem con el primer
    output_table.drop('AntennaID', axis=1, inplace=True)
    output_table.drop('count', axis=1, inplace=True)
    
    #para los datos faltantes dentro del Top10, relleno con -1s, que vendrian a ser los NaNs
    output_table.fillna(-1,inplace=True)
    
    #ojo aca que en el caso que entren hashes entonces no los va a poder convertir
    output_table = output_table.astype(int,copy=False)
    
    #agrego info de EPIDEMIC, asumiendo a alguien como epidemico si al antenna donde vive (la AntennaID_0) 
    #esta catalogada como EPIDEMIC
    test_table =test_table.join(antennas['EPIDEMIC'], on='AntennaID_0')
    
    
    #print(output_table.columns)
    #print("la tabla es:\n") 
    #print(output_table.head(5))
    #print("\n")
    
    #print(output_table.dytpes)
    #pd.to_numeric(output_table['EXPOSED'])
    
    #for i in range(3,22):
    #    pd.to_numeric(output_table[output_table.columns[i]]) 
    #print(output_table.dytpes)
    
    
    
    
    #aca termino guardando el output final pero solo para esos usuarios % pass ==group
    output_table.to_csv(output_file, index = False, 
                   header = False, mode='a',compression='gzip')
then = now
seconds = time.time() - then
print("total running time of script is %d " % seconds)

In [ ]:
#para observar la distribucion de veces que un usuario llama en un mes.
tabla['Target'].value_counts()[(tabla['Target'].value_counts()>5) & (tabla['Target'].value_counts() < 500)].hist(bins=100)

In [ ]:
sample['Target'].value_counts()[(sample['Target'].value_counts()>5) & (sample['Target'].value_counts() < 500)].hist(bins=35)

In [ ]:
#con esto extraigo el mapping user --> antennas used that month
user_antenna = tabla.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();
user_antenna2 = tabla.groupby('Target')['AntennaID'].apply(lambda x: x.tolist()).to_dict();

user_antenna_map_concat = defaultdict(list)

#Veo como concatenar dos "maps" distintos y dejando solo las antenas unicas..
for d in (user_antenna_map, user_antenna_map2): # podemos meter cuantos dicts queramos aca..
    for key, value in d.iteritems():
        #list(set()) elimina repetidos
        user_antenna_map_concat[key] = list(set(user_antenna_map_concat[key] + value))


In [ ]:
import subprocess
subprocess.call(" echo \"Hello World \" ",shell=True)

In [ ]:
os.listdir(os.getcwd())    

In [ ]:
os.chdir(rootdir)
os.walk('.').next()[1]

In [ ]:
os.chdir(rootdir)
print "testdata" in os.listdir(os.getcwd()) 
if "testdata" in os.listdir(os.getcwd()):
    os.system("mkdir testdata")

In [ ]:
os.getcwd()

In [ ]:
os.listdir(os.getcwd())

In [ ]:
!ls